In [12]:
%reload_ext autoreload
%autoreload 2

import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

code_dir = os.path.dirname(os.getcwd())
sys.path.append(code_dir)

from glob import glob
import pandas as pd
import numpy as np

import essentia.standard as es

from model.utils.audio_utils import load_audio

import IPython.display as ipd

from matplotlib import pyplot as plt

emb_dir = "../logs/emb/"

In [41]:
def analyze_result(df, idx, hop_size=0.5):

    row = df.iloc[idx]

    seq_len = (row['seq_len']+1) * hop_size

    gt_sequence = load_audio(row["gt_track_path"],
                            seg_start_sec=0.5*row["gt_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)

    pred_sequence = load_audio(row["pred_track_path"],
                            seg_start_sec=0.5*row["pred_start_segment"], 
                            seg_length_sec=seq_len,
                            normalize=False)

    print(f"GT: {os.path.basename(row['gt_track_path'])}")
    print(f"Seg: {row['gt_start_segment']}")
    ipd.display(ipd.Audio(gt_sequence, rate=8000, normalize=False))

    print(f"Pred: {os.path.basename(row['pred_track_path'])}")
    print(f"Seg: {row['pred_start_segment']}")
    ipd.display(ipd.Audio(pred_sequence, rate=8000, normalize=False))

In [3]:
dataset_dir  = "../../datasets/neural-audio-fp-dataset/"
music_dir = os.path.join(dataset_dir, "music")

test_dummy_dir = os.path.join(music_dir, "test-dummy-db-100k-full")
test_dummy_paths = glob(os.path.join(test_dummy_dir, "**/*.wav"), recursive=True)
print(len(test_dummy_paths))

test_query_dir = os.path.join(music_dir, "test-query-db-500-30s")

query_dir = os.path.join(test_query_dir, "query", "snr_00dB_10dB_1s")
query_paths = sorted(glob(os.path.join(query_dir, "**/*.wav"), recursive=True))
print(len(query_paths))

db_dir = os.path.join(test_query_dir, "db")
db_paths = sorted(glob(os.path.join(db_dir, "**/*.wav"), recursive=True))
print(len(db_paths))


100000
500
500


In [43]:
#analysis_path = "/home/oaraz/nextcore/fingerprinting/neural-audio-fp/logs/song_try/fma-random_amp-no_bg_ir_norm-bsz_128/100/float32/analysis.csv"
model_emb_path = "fma-old_sched-val_shift/100"
analysis_path = os.path.join(emb_dir, model_emb_path, "analysis.csv")

analysis_df = pd.read_csv(analysis_path)
print(analysis_df.shape)
analysis_df.head()

(6000, 7)


,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
0,14655,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.688491
1,14655,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.556114
2,14655,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.531633
3,21662,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,166,0.473909
4,21662,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,0.264480


In [15]:
# Correct track
analysis_df[analysis_df['gt_track_path'] == analysis_df['pred_track_path']]

,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
0,14655,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.688491
1,14655,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.556114
2,14655,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,/home/oaraz/nextcore/fingerprinting/datasets/n...,23,0.531633
4,21662,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,0.264480
5,21662,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,0.157989
...,...,...,...,...,...,...,...
5995,1097,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,35,/home/oaraz/nextcore/fingerprinting/datasets/n...,35,0.341360
5996,1097,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,35,/home/oaraz/nextcore/fingerprinting/datasets/n...,35,0.307958
5997,4538,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,54,/home/oaraz/nextcore/fingerprinting/datasets/n...,54,0.524681
5998,4538,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,54,/home/oaraz/nextcore/fingerprinting/datasets/n...,54,0.428825


In [24]:
# False track
false_track_df = analysis_df[analysis_df['gt_track_path'] != analysis_df['pred_track_path']]
false_track_df.head()

,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
3,21662,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,9,/home/oaraz/nextcore/fingerprinting/datasets/n...,166,0.473909
9,23837,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,469,0.579241
10,23837,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,34,0.430766
11,23837,5,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,870,0.323890
36,6108,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,31,/home/oaraz/nextcore/fingerprinting/datasets/n...,2167,0.614142


In [25]:
# Correct track, false segment
c_track_f_segment_df = analysis_df[(analysis_df['gt_track_path'] == analysis_df['pred_track_path']) & ((analysis_df['gt_start_segment'] != analysis_df['pred_start_segment']))]
c_track_f_segment_df.head()

,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
33,13264,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,48,/home/oaraz/nextcore/fingerprinting/datasets/n...,32,0.546977
51,7189,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,50,/home/oaraz/nextcore/fingerprinting/datasets/n...,49,0.640890
90,8438,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,2,0.641435
132,25567,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,29,0.849110
133,25567,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,29,0.811024


In [34]:
c_track_very_f_segment_df = analysis_df[(analysis_df['gt_track_path'] == analysis_df['pred_track_path']) & (np.abs(analysis_df['gt_start_segment'] - analysis_df['pred_start_segment']) > 1)]
c_track_very_f_segment_df.head()

,test_id,seq_len,gt_track_path,gt_start_segment,pred_track_path,pred_start_segment,score
33,13264,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,48,/home/oaraz/nextcore/fingerprinting/datasets/n...,32,0.546977
132,25567,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,29,0.849110
133,25567,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,20,/home/oaraz/nextcore/fingerprinting/datasets/n...,29,0.811024
199,22565,3,/home/oaraz/nextcore/fingerprinting/datasets/n...,27,/home/oaraz/nextcore/fingerprinting/datasets/n...,2,0.339990
270,26346,1,/home/oaraz/nextcore/fingerprinting/datasets/n...,32,/home/oaraz/nextcore/fingerprinting/datasets/n...,47,0.575992


In [40]:
analyze_result(c_track_very_f_segment_df, 25)

GT: 111866.wav
Seg: 49


Pred: 111866.wav
Seg: 17
